In [ ]:
import pandas as pd
import unidecode
import requests
from bs4 import BeautifulSoup
import multiprocessing as mp
import numpy as np


# Reading data and normalizing columns

In [ ]:
df = pd.read_excel("../../data/raw/ovsp_bdd_octubre.xlsx")
column_names_normalized = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.columns = column_names_normalized

# Quick and Dirty EDA: Confirming that I'm subsetting the correct number of elpitazo news

In [ ]:

# Confirming that I'm subsetting the correct number of elpitazo links
    # Result: 2483 el pitazo links
pitazo_mask = df.link_de_la_noticia.str.contains("https://elpitazo.net",na=False)
df_elpitazo = df[pitazo_mask]
# df.link_de_la_noticia[pitazo_mask].str.split("/", expand = True)[2].value_counts()

# Elpitazo webscraper

Common classes from `elpitazo`:

- Titlo de noticia `tdb-title-text`
- info general de la noticia `tdb-title-text`
- Texto completo de la noticia class="tdb-block-inner td-fix-index"
                            id="bsf_rt_marker"

## Testing with different classes

In [ ]:
# Testing different classes
# The easiest way to extract the news content is to extract all p tags and then clean them.

headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }

URL = "https://elpitazo.net/los-llanos/vecinos-de-varias-comunidades-de-acarigua-tienen-siete-anos-sin-agua/"
page = requests.get(URL, headers=headers, timeout=20)

soup = BeautifulSoup(page.content, "html.parser")

soup.find_all("h1", {"class": "tdb-title-text"}) # This snippet works well to find the news article title
# # soup.find_all("div", {"class": "tdb-block-inner td-fix-index"})
# soup.find_all("p", {"class": "tdb-block-inner td-fix-index"})

ls = []

tags_p = soup.find_all("p")
for tags in tags_p:
    
    if tags.has_attr('class') and tags['class'][0] in ['contacto-datos', 'text-white', '__cf_email__']:
        continue
        
    ls.append(tags.get_text())

' '.join(ls)


### Webscraper first pass

TODO: Use multiprocessing instead of pandas.apply. There are too many links to do it in a sequential fashion 

In [ ]:
def webscraper_elpitazo(url:str):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    
    page = requests.get(url, headers=headers, timeout=20)

    soup = BeautifulSoup(page.content, "html.parser")
    
    tags_p = soup.find_all("p")


    ls = [] # Temporary list
    for tags in tags_p:

        if tags.has_attr('class') and tags['class'][0] in ['contacto-datos', 'text-white', '__cf_email__']:
            continue

        ls.append(tags.get_text())

    complete_news = ' '.join(ls)
    
    return complete_news

In [ ]:
# Single URL Test
URL = "https://elpitazo.net/los-llanos/el-gas-domestico-en-acarigua-araure-cuesta-entre-10-y-20-dolares/"
webscraper_elpitazo(url=URL)

# pd.series text 
elpitazo_text = df_elpitazo.head().link_de_la_noticia.apply(webscraper_elpitazo)

## Multiprocessing implementation

In [ ]:
# TODO: Implement parallel processing. 

def parallelize_dataframe(df, func):
    num_processes = mp.cpu_count()
    
    df_split = np.array_split(df, num_processes)
    
    with mp.Pool(num_processes) as p:
        df = pd.concat(p.map(func, df_split))
    return df

parallelize_dataframe(df_elpitazo.head(),webscraper_elpitazo )